<a href="https://colab.research.google.com/github/lianavalentino/nu_capstone/blob/master/other/colab/Using%20the%20Natural%20Language%20API%20with%20Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

```text
SPDX-FileCopyrightText: 2023 Google LLC
SPDX-License-Identifier: Apache-2.0
```


# 🤯 Using the Natural Language API with Python

<center>
<table><tr><td>
<img src="https://github.com/GoogleCloudPlatform/devrel-demos/blob/main/other/colab/pics/natural_language_api.png?raw=1" style="height:200px" height="200" />
</td></tr></table>
<table><tr>
<td><a href="https://colab.research.google.com/github/GoogleCloudPlatform/devrel-demos/blob/main/other/colab/Using%20the%20Natural%20Language%20API%20with%20Python.ipynb">
<img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo" align="center"> Open in Colab
</a></td>
<td><a href="https://github.com/GoogleCloudPlatform/devrel-demos/blob/main/other/colab/Using%20the%20Natural%20Language%20API%20with%20Python.ipynb">
<img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo" align="center"> View on GitHub
</a></td>
</tr></table>
</center>

The [Natural Language API](https://cloud.google.com/natural-language/docs/) lets you extract information from unstructured text using Google machine learning. In this tutorial, you'll focus on using its Python client library to perform the following:

- Sentiment analysis
- Entity analysis
- Syntax analysis
- Content classification
- Text moderation (powered by [PaLM 2](https://blog.google/technology/ai/google-palm-2-ai-large-language-model))

This notebook requires a Google Cloud project:

- If needed, [create a new Google Cloud project](https://console.cloud.google.com/cloud-resource-manager).
- Make sure that billing is enabled for your project.
- It uses billable services but not should generate any cost (see the Natural Language API [free monthly thresholds](https://cloud.google.com/natural-language/pricing)).

It can run in autopilot mode:

- Launch "Run all".
- If the setup installs packages and restarts the kernel, launch "Run all" again.
- If requested, allow this notebook to access your Google credentials by signing in with your Google Cloud account.
- If requested, select your Google Cloud project and continue ("Run after").

> This port to a notebook was originally published on [Google Developers Codelabs](https://codelabs.developers.google.com/codelabs/cloud-natural-language-python3).


---

## ✔️ Setup


In [1]:
# @title 📦️ Packages (may restart) {display-mode: "form"}

# Dependencies (using Colab defaults for minimum versions)
# Assumption: later versions are backward compatible
PYTHON_MIN_VERSION = 3, 10
NOTEBOOK_PACKAGES = [
    ("pandas", "1.5.3"),
    ("Jinja2", "3.1.2"),
    ("ipywidgets", "7.7.1"),
]
NOTEBOOK_DEPENDENCIES = [
    (package, min_version_str, package.lower())
    for package, min_version_str in NOTEBOOK_PACKAGES
]
# Google Cloud APIs needed for this notebook
# - {API}.googleapis.com is the service name
# - google-cloud-{API} is its Python client library
GOOGLE_CLOUD_APIS = [
    ("language", "2.11.0"),
]
GOOGLE_CLOUD_SERVICES = [
    f"{API}.googleapis.com" for API, _ in GOOGLE_CLOUD_APIS
]
NOTEBOOK_DEPENDENCIES += [
    (f"google-cloud-{API}", min_version_str, f"google.cloud.{API}")
    for API, min_version_str in GOOGLE_CLOUD_APIS
]

import importlib.metadata
import sys

import packaging.version
from IPython.core.getipython import get_ipython

running_in_colab = "google.colab" in sys.modules
if running_in_colab:
    from google.colab import auth as colab_auth
restart_after_installation = running_in_colab


class StopExecution(Exception):
    def _render_traceback_(self):  # Suppress traceback in notebook
        return [self.args[0]] if self.args else None


def check_python_version():
    version = ".".join(map(str, sys.version_info[0:3]))
    if PYTHON_MIN_VERSION <= sys.version_info:
        print(f"✔️ Python: {version}")
        return
    required = ".".join(map(str, PYTHON_MIN_VERSION))
    raise StopExecution(f"❌ Python: {version} (version {required}+ is required)")


def check_packages():
    needed_packages = [
        package
        for package, min_version_str, lib in NOTEBOOK_DEPENDENCIES
        if need_package(package, min_version_str, lib)
    ]
    if not needed_packages:
        return
    requirements = " ".join(needed_packages)
    %pip install --upgrade $requirements --quiet
    if restart_after_installation:
        restart_kernel()


def need_package(package: str, min_version_str: str, lib: str) -> bool:
    min_version = packaging.version.parse(min_version_str)
    try:
        lib_version = packaging.version.parse(importlib.metadata.version(lib))
        if min_version <= lib_version:
            print(f"✔️ {package}=={lib_version!s}")
            return False
        print(f"📦️ {package} needs to be updated…")
    except importlib.metadata.PackageNotFoundError:
        print(f"📦️ {package} needs to be installed…")
    return True


def restart_kernel():
    if ipython := get_ipython():
        ipython.kernel.do_shutdown(True)
    raise StopExecution("❌ Restarting the kernel… Please run again")


def gcloud(command: str) -> list[str]:
    command = command.replace("'", '"')
    lines = !gcloud $command
    if any("ERROR:" in line for line in lines):
        raise StopExecution(lines.nlstr)
    return lines.list


check_python_version()
check_packages()

✔️ Python: 3.11.11
✔️ pandas==2.2.2
✔️ Jinja2==3.1.5
✔️ ipywidgets==7.7.1
✔️ google-cloud-language==2.16.0


In [2]:
# @title 🔑 Authentication {display-mode: "form"}


def get_active_account() -> str:
    lines = gcloud("config list --format 'value(core.account)'")
    return lines[0] if lines else ""


def is_authenticated() -> bool:
    return get_active_account() not in ["", "default"]


if "PYTHON_MIN_VERSION" not in globals():
    raise RuntimeWarning("❌ Please run the previous cell")

if not (authenticated := is_authenticated()):
    print(f"🔑 Authenticate to access your Google Cloud services")
    if running_in_colab:
        colab_auth.authenticate_user()
    else:
        gcloud("auth login --brief")
    authenticated = is_authenticated()

if authenticated:
    print(f"✔️ Authenticated")
else:
    raise StopExecution("❌ Could not authenticate")

🔑 Authenticate to access your Google Cloud services
✔️ Authenticated


In [3]:
# @title ⚙️ Project ID {display-mode: "form"}


def get_default_project_id() -> str:
    lines = gcloud("config list --format 'value(core.project)'")
    return lines[0] if lines else ""


def get_active_project_ids() -> list[str]:
    return gcloud("projects list --format 'value(projectId)'")


def update_project_id(project_id: str | None = None):
    if not project_id:
        raise StopExecution("❌ Please select your project")
    global PROJECT_ID
    if PROJECT_ID != project_id:
        PROJECT_ID = project_id
        if running_in_colab:
            colab_auth.authenticate_user(project_id=PROJECT_ID)
        else:
            print("⚙️ Revoking Application Default Credentials…")
            gcloud("auth application-default revoke --quiet")
            print("⚙️ Setting Application Default Credentials…")
            gcloud(f"auth application-default login --project {PROJECT_ID}")
            print("⚙️ Setting default project…")
            gcloud(f"config set project {PROJECT_ID}")
    print(f"✔️ PROJECT_ID: {PROJECT_ID}")


def show_project_ids(project_ids: list[str]):
    import ipywidgets

    @ipywidgets.interact(PROJECT_ID=project_ids)
    def on_project_id(PROJECT_ID: str | None = None):
        update_project_id(PROJECT_ID)


if PROJECT_ID := get_default_project_id():
    update_project_id(PROJECT_ID)
else:
    project_ids = get_active_project_ids()
    if len(project_ids) == 1:  # Single project (e.g. onboarding developer)
        update_project_id(project_ids[0])
    else:
        show_project_ids(project_ids)


interactive(children=(Dropdown(description='PROJECT_ID', options=('stock-prediction-450009', 'clean-sequencer-…

In [5]:
# @title 🔓 Project APIs {display-mode: "form"}


def get_enabled_services() -> list[str]:
    return gcloud("services list --enabled --format 'value(config.name)'")


if not PROJECT_ID:
    raise StopExecution("❌ PROJECT_ID is undefined")

if not GOOGLE_CLOUD_SERVICES:
    print(f"✔️ No specific API needed")
else:
    enabled_services = get_enabled_services()
    services_to_enable = [
        service for service in GOOGLE_CLOUD_SERVICES if service not in enabled_services
    ]
    if services_to_enable:
        api_or_apis = "APIs" if 1 < len(services_to_enable) else "API"
        spaced_services = " ".join(services_to_enable)
        print(f'🔓 Enabling {api_or_apis} "{spaced_services}"…')
        gcloud(f"services enable {spaced_services}")
        enabled_services = get_enabled_services()

    for service in GOOGLE_CLOUD_SERVICES:
        if service in enabled_services:
            print(f'✔️ API "{service}" is enabled')
        else:
            raise StopExecution(f'❌ Failed to enable API "{service}"')

✔️ API "language.googleapis.com" is enabled


In [6]:
# @title 🛠️ Helper functions {display-mode: "form"}

import pandas as pd
from IPython.display import display


def show_table(columns, data, formats=None, remove_empty_columns=False):
    df = pd.DataFrame(columns=columns, data=data)
    if remove_empty_columns:
        empty_cols = [col for col in df if df[col].eq("").all()]
        df.drop(empty_cols, axis=1, inplace=True)
    # Customize formatting
    styler = df.style
    if formats:
        styler.format(formats)
    # Left-align string columns
    df = df.convert_dtypes()
    str_cols = list(df.select_dtypes("string").keys())
    styler = styler.set_properties(subset=str_cols, **{"text-align": "left"})
    # Center headers
    styler.set_table_styles([{"selector": "th", "props": [("text-align", "center")]}])
    styler.hide()
    display(styler)


print(f"✔️ Helpers defined")

✔️ Helpers defined


---

## 🐍 Using the Python client library

You can use the Natural Language API in Python with the client library `google-cloud-language` and the following import:

In [7]:
from google.cloud import language_v1 as language


> Note: Version `v1` is Generally Available (GA). Version `v2` is currently in Preview.

---

## 1️⃣ Sentiment analysis

Sentiment analysis inspects the given text and identifies the prevailing emotional opinions within the text, especially to determine expressed sentiments as positive, negative, or neutral, both at the sentence and the document levels. It is performed with the `analyze_sentiment` method which returns an `AnalyzeSentimentResponse`.


In [8]:
from google.cloud import language_v1 as language


def analyze_text_sentiment(text: str) -> language.AnalyzeSentimentResponse:
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return client.analyze_sentiment(document=document)


def show_text_sentiment(response: language.AnalyzeSentimentResponse):
    columns = ["score", "sentence"]
    data = [(s.sentiment.score, s.text.content) for s in response.sentences]
    formats = {"score": "{:+.1f}"}
    print("At sentence level:")
    show_table(columns, data, formats)

    sentiment = response.document_sentiment
    columns = ["score", "magnitude", "language"]
    data = [(sentiment.score, sentiment.magnitude, response.language)]
    formats = {"score": "{:+.1f}", "magnitude": "{:.1f}"}
    print("")
    print("At document level:")
    show_table(columns, data, formats)

Now, let's perform an analysis:


In [10]:
# prompt: read in table from big query stock-prediction-450009.stock_data.earnings_call_transcripts. then create a widget to filter on ticker, year, quarter if wanted

from google.colab import auth
auth.authenticate_user()
import pandas as pd
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client(project='stock-prediction-450009')

# TODO(developer): Set table_id to the ID of the table to read.
table_id = 'stock-prediction-450009.stock_data.earnings_call_transcripts'

# Read the table into a Pandas DataFrame.
df = client.query(f'SELECT * FROM `{table_id}`').to_dataframe()


import ipywidgets as widgets
from IPython.display import display, clear_output

# Get unique values for filtering
tickers = df['ticker'].unique()
years = df['year'].unique()
quarters = df['quarter'].unique()


ticker_dropdown = widgets.Dropdown(
    options=tickers,
    description='Ticker:',
)

year_dropdown = widgets.Dropdown(
    options=years,
    description='Year:',
)

quarter_dropdown = widgets.Dropdown(
    options=quarters,
    description='Quarter:',
)


def on_button_clicked(b):
    clear_output(wait=True)
    display(ticker_dropdown, year_dropdown, quarter_dropdown, button)

    selected_ticker = ticker_dropdown.value
    selected_year = year_dropdown.value
    selected_quarter = quarter_dropdown.value

    filtered_df = df[(df['ticker'] == selected_ticker) &
                     (df['year'] == selected_year) &
                     (df['quarter'] == selected_quarter)]
    display(filtered_df)


button = widgets.Button(description="Filter")
button.on_click(on_button_clicked)

display(ticker_dropdown, year_dropdown, quarter_dropdown, button)


Dropdown(description='Ticker:', options=('UBER', 'ADSK', 'FTNT', 'AMAT', 'PLTR', 'AAPL', 'NVDA', 'CSCO', 'NOW'…

Dropdown(description='Year:', options=(2024, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023), value=2024…

Dropdown(description='Quarter:', options=('Q2', 'Q3', 'Q1', 'Q4'), value='Q2')

Button(description='Filter', style=ButtonStyle())

,ticker,year,quarter,datetime,speakers_list,transcript
0,UBER,2024,Q2,2024-08-06 08:00:00,"Ross Sandler,Prashanth Mahendra-Rajah,John Col...",Operator: Thank you and welcome to the Uber Q2...


In [11]:
# prompt: based on the selected widget filters above, create a new dataset. loop through that dataset and select the transcript field as text analyze sentiment analysis

import ipywidgets as widgets
from IPython.display import display, clear_output

# Assuming filtered_df is available from the previous code cell
# and contains the selected data based on widget filters.

def analyze_transcripts(df):
    """Analyzes sentiment of transcripts in a DataFrame."""

    client = language.LanguageServiceClient()

    for index, row in df.iterrows():
        transcript = row['transcript']
        if isinstance(transcript, str) and transcript.strip():  # Check for empty or non-string values
            document = language.Document(
                content=transcript,
                type_=language.Document.Type.PLAIN_TEXT
            )

            response = client.analyze_sentiment(request={'document': document})
            sentiment = response.document_sentiment

            print(f"--- Transcript {index + 1} ---")
            print(f"  Score: {sentiment.score}")
            print(f"  Magnitude: {sentiment.magnitude}")
            print("-" * 20)
        else:
            print(f"--- Transcript {index + 1} ---")
            print("  Transcript is empty or not a string, skipping analysis.")
            print("-" * 20)

# Example usage (assuming button click event):
def on_button_clicked(b):
    clear_output(wait=True)
    display(ticker_dropdown, year_dropdown, quarter_dropdown, button)

    selected_ticker = ticker_dropdown.value
    selected_year = year_dropdown.value
    selected_quarter = quarter_dropdown.value

    filtered_df = df[(df['ticker'] == selected_ticker) &
                     (df['year'] == selected_year) &
                     (df['quarter'] == selected_quarter)]
    display(filtered_df)
    analyze_transcripts(filtered_df) # Call the analysis function

button = widgets.Button(description="Filter and Analyze")
button.on_click(on_button_clicked)

display(ticker_dropdown, year_dropdown, quarter_dropdown, button)


Dropdown(description='Ticker:', options=('UBER', 'ADSK', 'FTNT', 'AMAT', 'PLTR', 'AAPL', 'NVDA', 'CSCO', 'NOW'…

Dropdown(description='Year:', options=(2024, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023), value=2024…

Dropdown(description='Quarter:', options=('Q2', 'Q3', 'Q1', 'Q4'), value='Q2')

Button(description='Filter and Analyze', style=ButtonStyle())

,ticker,year,quarter,datetime,speakers_list,transcript
0,UBER,2024,Q2,2024-08-06 08:00:00,"Ross Sandler,Prashanth Mahendra-Rajah,John Col...",Operator: Thank you and welcome to the Uber Q2...


--- Transcript 1 ---
  Score: 0.20000000298023224
  Magnitude: 112.19999694824219
--------------------


In [9]:
# Input
text = "Python is a very readable language, which makes it easy to understand and maintain code. It's simple, very flexible, easy to learn, and suitable for a wide variety of tasks. One disadvantage is its speed: it's not as fast as some other programming languages."  # @param {type:"string"}

# Send a request to the API
analyze_sentiment_response = analyze_text_sentiment(text)

# Show the results
show_text_sentiment(analyze_sentiment_response)

At sentence level:


score,sentence
+0.8,"Python is a very readable language, which makes it easy to understand and maintain code."
+0.9,"It's simple, very flexible, easy to learn, and suitable for a wide variety of tasks."
-0.4,One disadvantage is its speed: it's not as fast as some other programming languages.



At document level:


score,magnitude,language
+0.4,2.2,en


Notes:

- For information on which languages are supported by the Natural Language API, see [Language Support](https://cloud.google.com/natural-language/docs/languages#sentiment_analysis).
- The `score` of the sentiment ranges between -1.0 (negative) and 1.0 (positive) and corresponds to the overall sentiment from the given information.
- The `magnitude` of the sentiment ranges from 0.0 to +infinity and indicates the overall strength of sentiment from the given information. The more information provided, the higher the magnitude.
- For more information on how to interpret the `score` and `magnitude` sentiment values included in the analysis, see [Interpreting sentiment analysis values](https://cloud.google.com/natural-language/docs/basics#interpreting_sentiment_analysis_values).
- Each API response returns the document automatically-detected language (in ISO-639-1). It is shown here and will be skipped in the next analysis examples.


---

## 2️⃣ Entity analysis

Entity analysis inspects the given text for known entities (proper nouns such as public figures, landmarks, etc.), and returns information about those entities. It is performed with the `analyze_entities` method which returns an `AnalyzeEntitiesResponse`.


In [12]:
from google.cloud import language_v1 as language


def analyze_text_entities(text: str) -> language.AnalyzeEntitiesResponse:
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return client.analyze_entities(document=document)


def show_text_entities(response: language.AnalyzeEntitiesResponse):
    columns = ("name", "type", "salience", "mid", "wikipedia_url")
    data = (
        (
            entity.name,
            entity.type_.name,
            entity.salience,
            entity.metadata.get("mid", ""),
            entity.metadata.get("wikipedia_url", ""),
        )
        for entity in response.entities
    )
    formats = {"salience": "{:.1%}"}
    show_table(columns, data, formats)

Now, let's perform an analysis:


In [13]:
# Input
text = "Guido van Rossum is best known as the creator of Python, which he named after the Monty Python comedy troupe. He was born in Haarlem, Netherlands."  # @param {type:"string"}

# Send a request to the API
analyze_entities_response = analyze_text_entities(text)

# Show the results
show_text_entities(analyze_entities_response)

name,type,salience,mid,wikipedia_url
Guido van Rossum,PERSON,49.8%,/m/01h05c,https://en.wikipedia.org/wiki/Guido_van_Rossum
Python,ORGANIZATION,38.4%,/m/05z1_,https://en.wikipedia.org/wiki/Python_(programming_language)
creator,PERSON,5.1%,,
Monty Python,PERSON,3.2%,/m/04sd0,https://en.wikipedia.org/wiki/Monty_Python
comedy troupe,PERSON,1.6%,,
Haarlem,LOCATION,1.0%,/m/0h095,https://en.wikipedia.org/wiki/Haarlem
Netherlands,LOCATION,0.7%,/m/059j2,https://en.wikipedia.org/wiki/Netherlands


Notes:

- For information on which languages are supported by this method, see [Language Support](https://cloud.google.com/natural-language/docs/languages#entity_analysis).
- The `type` of the entity is an enum that lets you classify or differentiate entities. For example, this can help distinguish the similarly named entities _“T.E. Lawrence”_ (a `PERSON`) from _“Lawrence of Arabia”_ (the film) (tagged as a `WORK_OF_ART`). See [`Entity.Type`](https://cloud.google.com/python/docs/reference/language/latest/google.cloud.language_v1.types.Entity.Type).
- The entity `salience` indicates the importance or relevance of this entity to the entire document text. This score can assist information retrieval and summarization by prioritizing salient entities. Scores closer to 0.0 are less important, while scores closer to 1.0 are highly important.
- For more information, see [Entity analysis](https://cloud.google.com/natural-language/docs/basics#entity_analysis).
- You can also combine both entity analysis and sentiment analysis with the `analyze_entity_sentiment` method. See [Entity sentiment analysis](https://cloud.google.com/natural-language/docs/basics#entity_analysis).


---

## 3️⃣ Syntax analysis

Syntax analysis extracts linguistic information, breaking up the given text into a series of sentences and tokens (generally based on word boundaries), providing further analysis on those tokens. It is performed with the `analyze_syntax` method which returns an `AnalyzeSyntaxResponse`.


In [ ]:
from google.cloud import language_v1 as language


def analyze_text_syntax(text: str) -> language.AnalyzeSyntaxResponse:
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return client.analyze_syntax(document=document)


def get_token_info(token: language.Token | None) -> list[str]:
    parts = [
        "tag",
        "aspect",
        "case",
        "form",
        "gender",
        "mood",
        "number",
        "person",
        "proper",
        "reciprocity",
        "tense",
        "voice",
    ]
    if not token:
        return ["token", "lemma"] + parts

    text = token.text.content
    lemma = token.lemma if token.lemma != token.text.content else ""
    info = [text, lemma]
    for part in parts:
        pos = token.part_of_speech
        info.append(getattr(pos, part).name if part in pos else "")

    return info


def show_text_syntax(response: language.AnalyzeSyntaxResponse):
    tokens = len(response.tokens)
    sentences = len(response.sentences)
    columns = get_token_info(None)
    data = (get_token_info(token) for token in response.tokens)

    print(f"Analyzed {tokens} token(s) from {sentences} sentence(s)")
    show_table(columns, data, remove_empty_columns=True)

Now, let's perform an analysis:


In [ ]:
# Input
text = "Guido van Rossum is best known as the creator of Python. He was born in Haarlem, Netherlands."  # @param {type:"string"}

# Send a request to the API
analyze_syntax_response = analyze_text_syntax(text)

# Show the results
show_text_syntax(analyze_syntax_response)

There are multiple benefits to extracting the syntax information. One of them is to extract the lemmas. A `lemma` contains the "root" word upon which this token is based, which allows you to manage words with their canonical forms.

If you dive deeper into the response insights, you'll also find the relationships between the tokens. Here is a visual interpretation showing the complete syntax analysis for this example:

![Syntax Analysis](https://github.com/GoogleCloudPlatform/devrel-demos/blob/main/other/colab/pics/natural_language_syntax.png?raw=1)

> This is a screenshot from the online [Natural Language demo](https://cloud.google.com/natural-language/#natural-language-api-demo) with which you can create your own parse trees.

For more information, see the following:

- [`language.AnalyzeSyntaxResponse`](https://cloud.google.com/python/docs/reference/language/latest/google.cloud.language_v1.types.AnalyzeSyntaxResponse)
- [Language Support](https://cloud.google.com/natural-language/docs/languages#syntactic_analysis)
- [Syntactic analysis](https://cloud.google.com/natural-language/docs/basics#syntactic_analysis)
- [Morphology & Dependency Trees](https://cloud.google.com/natural-language/docs/morphology)


---

## 4️⃣ Content classification

Content classification analyzes a document and returns a list of content categories that apply to the text found in the document. It is performed with the `classify_text` method which returns a `ClassifyTextResponse`.


In [ ]:
from google.cloud import language_v1 as language


def classify_text(text: str) -> language.ClassifyTextResponse:
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return client.classify_text(document=document)


def show_text_classification(response: language.ClassifyTextResponse):
    columns = ["category", "confidence"]
    data = ((category.name, category.confidence) for category in response.categories)
    formats = {"confidence": "{:.0%}"}
    show_table(columns, data, formats)

Now, let's perform an analysis:


In [ ]:
# Input
text = "Python is an interpreted, high-level, general-purpose programming language. Created by Guido van Rossum and first released in 1991, Python's design philosophy emphasizes code readability with its notable use of significant whitespace."  # @param {type:"string"}

# Send a request to the API
classify_text_response = classify_text(text)

# Show the results
show_text_classification(classify_text_response)

> Important: You must supply a text block (document) with at least twenty tokens.

For more information, see the following docs:

- [`ClassifyTextResponse`](https://cloud.google.com/python/docs/reference/language/latest/google.cloud.language_v1.types.ClassifyTextResponse)
- [Language Support](https://cloud.google.com/natural-language/docs/languages#content_classification)
- [Content Classification](https://cloud.google.com/natural-language/docs/basics#content-classification)


---

## 5️⃣ Text moderation

Powered by Google's latest [PaLM 2](https://blog.google/technology/ai/google-palm-2-ai-large-language-model) foundation model, text moderation identifies a wide range of harmful content, including hate speech, bullying, and sexual harassment. It is performed with the `moderate_text` method which returns a `ModerateTextResponse`.


In [ ]:
from google.cloud import language_v1 as language


def moderate_text(text: str) -> language.ModerateTextResponse:
    client = language.LanguageServiceClient()
    document = language.Document(
        content=text,
        type_=language.Document.Type.PLAIN_TEXT,
    )
    return client.moderate_text(document=document)


def show_text_moderation(response: language.ModerateTextResponse):
    def confidence(category: language.ClassificationCategory) -> float:
        return category.confidence

    columns = ["category", "confidence"]
    categories = response.moderation_categories
    sorted_categories = sorted(categories, key=confidence, reverse=True)
    data = ((category.name, category.confidence) for category in sorted_categories)
    formats = {"confidence": "{:.0%}"}
    show_table(columns, data, formats)

Now, let's perform an analysis:


In [ ]:
# Input
text = "I have to read Ulysses by James Joyce and am a little over halfway through. I hate it. What a pile of garbage!"  # @param {type:"string"}

# Send a request to the API
moderate_text_response = moderate_text(text)

# Show the results
show_text_moderation(moderate_text_response)

For more information, see the following docs:

- [`ModerateTextResponse`](https://cloud.google.com/python/docs/reference/language/latest/google.cloud.language_v1.types.ModerateTextResponse)
- [Language Support](https://cloud.google.com/natural-language/docs/languages#content_classification)
- [Moderating Text](https://cloud.google.com/natural-language/docs/moderating-text)


---

## 🎉 Congratulations

You learned how to use the Natural Language API with Python!

<center>
<table><tr><td>
<img src="https://github.com/GoogleCloudPlatform/devrel-demos/blob/main/other/colab/pics/natural_language_api.png?raw=1" style="height:200px;" height="200" />
</td></tr></table>
<table><tr>
